- [문제 링크](https://www.acmicpc.net/problem/9019)
- 취할 수 있는 액션이 4개이며 4자리수로 표시됨(DSLR)
    - D : 주어진 수를 2배로 하며, 9999보다 커지면 10000으로 나눈 나머지 값임
    - S : 주어진 수 - 1을 저장하며, 0이 되면 9999가 저장됨
    - L : 각 자릿수를 1칸 왼쪽으로 옮겨 저장함
    - R : 각 자릿수를 1칸 오른쪽으로 옮겨 저장함. 
- 2개의 수가 주어지고, 이 2개의 수에 대한 **최소한의 명령어**를 출력하면 됨

## 풀이

- 내가 푼 코드는 시간 초과가 떴음 
- 너무 길게 작성하기도 했어서 틀리면 바로 정답지 찾아볼 생각이었고
- [요 링크](https://my-coding-notes.tistory.com/282)를 참조했다.
- 이런 **간단하게 할 수 있는 팁들은 꼭 숙지**해두자! : 정해진 것에 너무 얽매일 필요 없다

> 1. 경로를 구현하는 방식  
> - 나는 방문 배열에 경로를 넣어놔야겠다는 생각을 했는데, Queue와 문자열 자료구조를 이용해서 더 간단하게 구현할 수 있다.  
> - **생각을 못했던 것 : Queue에 경로를 표시하면서 BFS를 쓴다는 것과, 문자열을 쓴다는 것**
> - 역추적을 하지 않고 정답이 나온다는 점에서도 빠르다


> 2. `L, R` 명령어 : **배열로 구현할 필요 없다.**  
> - `L`은 결국 기존 숫자 * 10한 값에 가장 앞자리 숫자를 뒤에 붙인 거임 : N * 10을 한 다음, 후자는 N // 1000으로 구할 수 있고 4자리 숫자니까 % 10000으로 맞춰준다
> - `R`도 비슷한 원리임 : 기존 N에서 10으로 나눈 나머지에 1000을 곱하면 맨 앞자리가 되고, 기존 N의 앞 3자리가 뒤로 와야 하므로 10으로 나눈 몫을 더하면 됨  
(ex : `1234` -> `4(1234 % 10) * 1000 + 234(1234 % 1000)`) 

In [46]:
# 이 풀이도 Python3는 시간초과가 뜨고 PyPy3로 돌려야 함
from collections import deque
import sys
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

for _ in range(int(input())):
    A, B = map(int, input().split())
    q = deque()
    q.append((A, "")) # 지나온 경로를 Queue를 이용해서 구현한다.
    visit = [False] * 10000
    
    while q:
        num, path = q.popleft()
        visit[num] = True
        if num == B:
            print(path)
            break
        
        # D
        num2 = (2 * num) % 10000
        if not visit[num2]:
            q.append((num2, path+"D"))
            visit[num2] = True
        
        # S
        # -1 % 10000 = -1 * 10000 + 9999이므로!
        num2 = (num - 1) % 10000 
        if not visit[num2]:
            q.append((num2, path + "S"))
            visit[num2] = True
            
        # L
        num2 = (10 * num + (num // 1000)) % 10000
        if not visit[num2]:
            q.append((num2, path + "L"))
            visit[num2] = True
            
        # R
        num2 = (num % 10 * 1000 + num // 10) % 10000
        if not visit[num2]:
            q.append((num2, path + "R"))
            visit[num2] = True

LL
L
DDDD


- 4개의 숫자를 따로 가지고 있고, 연산 결과를 큐에 저장
- 방문 리스트도 구현 가능할 거고..
- 일단 이전 인덱스를 표시하는데, 어떤 액션으로 왔는지를 표시해야 할 수도 있음

### 역추적 관련
#### D
- 만약 2가 있다고 치자
- 이 2는 1 * 2에서 온 건지 5001 * 2 에서 온 건지 그냥은 알 수 없음
- 따라서 모든 짝수에 대해 2가지 케이스로 나눠서 생각한다
    - 1. 그냥 2로 나눈 게 이전 값이거나
    - 2. 10000을 더한 뒤 2로 나눈 게 이전 값이거나

#### 나머지는 크게 어렵진 않은 듯?
- S는 1 더해주면 되고 9999면 0으로 보내면 됨
- L, R도 다시 뒤집어보면 됨

#### 모든 케이스에서 visited[인덱스] = visited[다음 인덱스] - 1 확인
- 행동 값을 저장해둬야 할까?
- 역추적을 하는 근거가 4개의 행동이란 말임?
- 그 4개의 행동을 거꾸로 했을 때 방문 순서를 거슬러 오를 수 있다는 건 설령 그쪽으로 오지 않았더라도 거꾸로 갈 수 있다는 말이기도 한 거잖아?

In [ ]:
# 시간 초과 : Python3, PyPy3 모두
import sys
from collections import deque

sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline
N = int(input())
for _ in range(N):
    A, B = map(int, input().split())

    INF = int(1e9)
    # 방문 순서와 이전에 취한 행동을 저장하겠음
    visited = [INF for _ in range(10000)] # 0 ~ 9999까지의 인덱스를 사용함

    # A -> B로 가는 BFS 부분
    dq = deque([])
    dq.append(A)
    visited[A] = 0 # 방문 순서
    # visited[A][1] = 0 # 이전 인덱스에서 취한 행동
    
    while dq:
        now_n = dq.popleft()
        
        if now_n == B: # B라면 역추적을 시작
            ans = []
            now_order = visited[now_n]
            
            while now_order > 0:
                temp = now_n
                now_n_lst = deque([])
                x = 1000
                
                while x > 0:
                    b = temp // x
                    now_n_lst.append(b)
                    temp = temp % x
                    x = x // 10
                
                
                for i in range(4):
                    if i == 0:
                        before_n1 = now_n // 2
                        before_n2 = (now_n + 10000) // 2
                        if visited[before_n1] == now_order - 1:
                            now_n = before_n1
                            ans.append('D')
                            now_order -= 1
                            break
                        elif visited[before_n2] == now_order - 1:
                            now_n = before_n2
                            ans.append('D')
                            now_order -= 1
                            break
                    
                    elif i == 1:
                        if now_n == 0:
                            before_n = 9999
                        else:
                            before_n = now_n - 1
                    
                        if visited[before_n] == now_order - 1:
                            now_n = before_n
                            ans.append('S')
                            now_order -= 1
                            break
                            
                    elif i == 2:
                        tmp_lst = now_n_lst.copy()
                        tmp_lst.rotate(1)
                        before_n = int("".join(map(str, tmp_lst)))
                        if visited[before_n] == now_order - 1:
                            now_n = before_n
                            ans.append('L')
                            now_order -= 1
                            break
                            
                    elif i == 3:
                        tmp_lst = now_n_lst.copy()
                        tmp_lst.rotate(-1)
                        before_n = int("".join(map(str, tmp_lst)))
                        if visited[before_n] == now_order - 1:
                            now_n = before_n
                            ans.append('R')
                            now_order -= 1
                            break
            
            print("".join(ans[::-1]))
            break
            
        temp = now_n
        x = 1000
        now_n_lst = deque([]) # l, r 때문에 만들어놓음
        
        # 뗴어낸 숫자를 4개로 쪼갬
        while x > 0:
            d_n = temp // x
            now_n_lst.append(d_n)
            temp = temp % x
            x = x // 10
        
        # 4개의 숫자에 대해 4개의 액션을 취함
        
        for i in range(4):
            if i == 0:
                new_n = (now_n * 2) % 10000
                order = 'D'
            elif i == 1:
                if now_n == 0:
                    new_n = 9999
                else:
                    new_n = now_n - 1
                order = 'S'
            elif i == 2:
                tmp_lst = now_n_lst.copy()
                tmp_lst.rotate(-1)
                new_n = int("".join(map(str, tmp_lst)))
                order = 'L'
                
            elif i == 3:
                tmp_lst = now_n_lst.copy()
                tmp_lst.rotate(1)
                new_n = int("".join(map(str, tmp_lst)))
                order = 'R'
            
            if visited[new_n] == INF:
                dq.append(new_n)
                visited[new_n] = visited[now_n] + 1
            
        
#         if visited[new_n1][0] == INF:
#             dq.append(new_n1)
#             visited[new_n1] = visited[now_n] + 1
#             # visited[new_n1][1] = 'D'
        
#         # 2. S : n-1을 저장한다. 0이라면 9999가 저장된다.
#         if now_n == 0:
#             new_n2 = 9999
#         else: # now_n이 음수 값이 오지 않게 해야 함
#             new_n2 = (now_n - 1)
#         if visited[new_n2] == INF:
#             dq.append(new_n2)
#             visited[new_n2] = visited[now_n] + 1
#             # visited[new_n2][1] = 'S'
            
#         # 3. l : 각 자릿수를 왼쪽으로 돌린다
#         tmp_lst = now_n_lst.copy()
#         tmp_lst.rotate(-1)
#         new_n3 = int("".join(map(str, tmp_lst)))
#         if visited[new_n3] == INF:
#             dq.append(new_n3)
#             visited[new_n3] = visited[now_n] + 1
#             # visited[new_n3][1] = 'L'
        
#         # 4. r : 오른쪽
#         tmp_lst = now_n_lst.copy()
#         tmp_lst.rotate(1)
#         new_n4 = int("".join(map(str, tmp_lst)))
#         if visited[new_n4] == INF:
#             dq.append(new_n4)
#             visited[new_n4] = visited[now_n] + 1
            # visited[new_n4][1] = 'R'
            
